In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
#import matplotlib.image as img
#import seaborn as sns
#import pickle

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
#import itertools
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import nltk
import numpy as np
from bs4 import BeautifulSoup
import re

from sklearn import metrics
from sklearn import tree
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import time



from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# cd /content/drive/My\ Drive/Colab\ Notebooks

In [3]:
cd /content/drive/My\ Drive/Colab\ Notebooks/dataset/input

/content/drive/My Drive/Colab Notebooks/dataset/input


## Load Data

In [0]:
# test = pd.read_csv('./test_yelp.csv')
data = pd.read_csv('./yelp2013.csv')

In [0]:
msk = np.random.rand(len(data)) < 0.9
train = data[msk]
test = data[~msk]

## Preprocess

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
stop = stopwords.words('english')

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def preprocess(dataset):
    dataset.apply(lambda x:" ".join(x.lower() for x in x.split()))
    dataset.str.replace('[^\w\s]','')
    dataset.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    dataset.apply(lambda x: " ".join([lm.lemmatize(word) for word in x.split()]))
    
    return dataset

## Data Split

In [0]:
Y_train = train['stars']
X_train = train['text']
Y_test = test['stars']
X_test = test['text']

In [0]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)

In [0]:
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

## Delete data

In [0]:
del data
del train
del test

##Logistic Regression

In [0]:
logreg = Pipeline([
    ('vect', CountVectorizer(binary = True)),
    ('tfidf', TfidfTransformer()),
    ('norm', Normalizer()),
    ('clf', LogisticRegression(C=10)),
])

In [22]:
seed = 130
start = time.time()
params_log = {"vect__ngram_range": [(1,1),(1,2),(2,2)],
              "vect__binary":[True, False],
              "vect__max_features":[50000,100000],
              "clf__penalty": ["l2"]}
logreg_cv = GridSearchCV(logreg, param_grid = params_log, cv=2, verbose = 10, n_jobs = -1)
logreg_cv.fit(X_train, Y_train)
end = time.time()
running_time = end-start
print("Running time is ",running_time)


Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  3.4min
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 23.0min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 54.9min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 59.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 59.2min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silen

Running time is  4325.974917888641


In [0]:
y_pred = logreg_cv.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

0.6921380461345891